# Which Language is?
## Lesson 4 - Student 211039312
## Danilo Carvalho Antunes
## Github: Danilo-Carvalho-Antunes
## Introduction
The main objetive of it is to create a model of machine learning capable of predict the language of the sentence in English, French, Portugeese, Russian, Sweedish and Unknow. To do this is necessary to use some techniques of deep learning.

It's made in jupyter, Hugging Face and using Gradio,Transformers,Numpy,Pandas and Torch

# Step 1 - downloading dependecies and the dataset

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE _KERNEL_RUN_TYPE', '')

In [132]:
creds = ''

In [133]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [134]:
path = Path('/kaggle/input/language-detection')

In [135]:
!ls {path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'Language Detection.csv'


# Step 2 - Cleaning and adapting the Dataset

In [136]:
import pandas as pd

In [137]:
df = pd.read_csv(path/'Language Detection.csv')
classes_a_remover = ['German', 'Italian','Malayalam','Danish','Kannada','Tamil','Spanish','Dutch','Greek','Arabic','Turkish','Hindi']
df = df[~df['Language'].isin(classes_a_remover)]
df = df.sample(2500)

In [138]:
df

,Text,Language
764,[4][5] In its application across business prob...,English
1289,permission.,English
2577,Nenhum artigo ou o seu conteúdo é propriedade ...,Portugeese
944,"Typically, artificial neurons are aggregated i...",English
3879,"Privatisé, car les algorithmes d’AA constituen...",French
...,...,...
6498,Береги себя.,Russian
619,The Wikipedia project sought the switch to the...,English
6215,был издан DVD польского раздела.,Russian
2868,[209] Em abril de 2010 Larry Sanger escreveu u...,Portugeese


In [139]:
df.describe()

,Text,Language
count,2500,2500
unique,2490,5
top,Боже ты мой.,English
freq,2,766


In [140]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [141]:
df.head()

,Text,Language
764,[4][5] In its application across business prob...,English
1289,permission.,English
2577,Nenhum artigo ou o seu conteúdo é propriedade ...,Portugeese
944,"Typically, artificial neurons are aggregated i...",English
3879,"Privatisé, car les algorithmes d’AA constituen...",French


In [142]:
df['Language'].value_counts()

Language
English       766
French        563
Portugeese    414
Sweedish      388
Russian       369
Name: count, dtype: int64

In [143]:
import numpy as np

In [144]:
mapping = {
    "English":np.log(2.00),
"French":np.log(3.00),
"Portugeese":np.log(4.00),
"Russian":np.log(5.00),
"Sweedish":np.log(6.00),
}
df['Language'] = df['Language'].map(mapping)

In [145]:
df['Language'].value_counts()

Language
0.693147    766
1.098612    563
1.386294    414
1.791759    388
1.609438    369
Name: count, dtype: int64

In [146]:
import re
def remove_accenttuation_punctuation(text):
    text = re.sub(r'"', '', text)
    text = re.sub(r'``', '', text)
    text = re.sub(r'«', '', text)
    text = re.sub(r'»', '', text)
    text = re.sub(r'؟', '', text)
    return text.strip()

df['Text'] = df['Text'].apply(remove_accenttuation_punctuation)

In [147]:
df['Text'].head()

764     [4][5] In its application across business prob...
1289                                          permission.
2577    Nenhum artigo ou o seu conteúdo é propriedade ...
944     Typically, artificial neurons are aggregated i...
3879    Privatisé, car les algorithmes d’AA constituen...
Name: Text, dtype: object

In [148]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [149]:
ds

Dataset({
    features: ['Text', 'Language', '__index_level_0__'],
    num_rows: 2500
})

# Step 3 - Pre-trained model and Tokenization

In [150]:
model_nm = 'microsoft/deberta-v3-small' 

In [151]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [152]:
def tok_func(x): return tokz(x["Text"])

In [153]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [154]:
row = tok_ds[0]
row['Text'], row['input_ids'][:10]

('[4][5] In its application across business problems, machine learning is also referred to as predictive analytics.',
 [1, 647, 554, 592, 2550, 524, 592, 344, 359, 1055])

In [155]:
tokz.vocab['Nature']

55397

In [156]:
tok_ds = tok_ds.rename_columns({'Language':'labels'})

# Step 4 - Training data and Validation data

In [157]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['Text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1875
    })
    test: Dataset({
        features: ['Text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 625
    })
})

In [158]:
eval_df = pd.read_csv('/kaggle/input/language-detection/Language Detection.csv')
classes_a_remover = ['German', 'Italian','Malayalam','Danish','Kannada','Tamil','Spanish','Dutch','Greek','Arabic','Turkish','Hindi']
eval_df = eval_df[~eval_df['Language'].isin(classes_a_remover)]
eval_df = eval_df.sample(80,random_state=42)
eval_df['Language'] = eval_df['Language'].map(mapping)
eval_df

,Text,Language
764,[4][5] In its application across business prob...,0.693147
1289,permission.,0.693147
2577,Nenhum artigo ou o seu conteúdo é propriedade ...,1.386294
944,"Typically, artificial neurons are aggregated i...",0.693147
3879,"Privatisé, car les algorithmes d’AA constituen...",1.098612
...,...,...
8909,inga problem.,1.791759
8440,Bidragsgivare till Wikipedias politikartiklar ...,1.791759
6671,"вот попробуйте золотого картофеля, он действит...",1.609438
683,"On September 28, 2007, Italian politician Fran...",0.693147


In [159]:
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [160]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [161]:
from transformers import TrainingArguments,Trainer

In [162]:
bs = 32
epochs = 4

In [163]:
lr = 8e-5

In [164]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [165]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [166]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.043732,0.938760
2,No log,0.009844,0.978074
3,No log,0.015712,0.986250
4,No log,0.009666,0.987261


TrainOutput(global_step=120, training_loss=0.10244858264923096, metrics={'train_runtime': 82.6827, 'train_samples_per_second': 90.708, 'train_steps_per_second': 1.451, 'total_flos': 267492657301320.0, 'train_loss': 0.10244858264923096, 'epoch': 4.0})

# Step 5 - Testing the model

In [167]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds[:80]

array([[0.71038067],
       [0.84315598],
       [1.42857873],
       [0.69334364],
       [1.17441952],
       [1.47009432],
       [1.93476176],
       [1.72892439],
       [0.69824702],
       [1.85302639],
       [1.391536  ],
       [0.69414222],
       [1.94172287],
       [0.69912624],
       [1.71475899],
       [0.69367081],
       [0.69384676],
       [1.93909204],
       [0.73264372],
       [1.42605257],
       [1.68224609],
       [0.70787221],
       [0.71099889],
       [0.77252495],
       [0.700535  ],
       [1.14109933],
       [1.94691718],
       [1.47724676],
       [1.18856823],
       [1.93844068],
       [0.70486826],
       [1.7006942 ],
       [0.69553977],
       [0.69658923],
       [1.94504988],
       [1.41302609],
       [1.9335357 ],
       [0.69988698],
       [1.16723251],
       [0.70580578],
       [1.71720946],
       [1.16502178],
       [1.12431037],
       [1.1483804 ],
       [1.72458601],
       [1.94574273],
       [0.69739926],
       [1.084

In [168]:
mapping = {
    2:"English",
3:"French",
4:"Portugeese",
5:"Russian",
6:"Sweedish",
7:"unknow"
}
preds = [mapping[round(np.exp(num[0]))] for num in preds.tolist()]
preds

['English',
 'English',
 'Portugeese',
 'English',
 'French',
 'Portugeese',
 'unknow',
 'Sweedish',
 'English',
 'Sweedish',
 'Portugeese',
 'English',
 'unknow',
 'English',
 'Sweedish',
 'English',
 'English',
 'unknow',
 'English',
 'Portugeese',
 'Russian',
 'English',
 'English',
 'English',
 'English',
 'French',
 'unknow',
 'Portugeese',
 'French',
 'unknow',
 'English',
 'Russian',
 'English',
 'English',
 'unknow',
 'Portugeese',
 'unknow',
 'English',
 'French',
 'English',
 'Sweedish',
 'French',
 'French',
 'French',
 'Sweedish',
 'unknow',
 'English',
 'French',
 'unknow',
 'Portugeese',
 'Sweedish',
 'unknow',
 'Russian',
 'French',
 'Russian',
 'Russian',
 'English',
 'unknow',
 'English',
 'English',
 'English',
 'Portugeese',
 'unknow',
 'Portugeese',
 'French',
 'Sweedish',
 'French',
 'unknow',
 'English',
 'English',
 'English',
 'French',
 'Portugeese',
 'English',
 'English',
 'French',
 'unknow',
 'Sweedish',
 'English',
 'unknow']

In [169]:
comp = pd.read_csv('/kaggle/input/language-detection/Language Detection.csv')
comp = comp[~comp['Language'].isin(classes_a_remover)]
comp = comp.sample(80,random_state=42)
comp['Preds'] = preds
comp

,Text,Language,Preds
764,[4][5] In its application across business prob...,English,English
1289,permission.,English,English
2577,Nenhum artigo ou o seu conteúdo é propriedade ...,Portugeese,Portugeese
944,"Typically, artificial neurons are aggregated i...",English,English
3879,"Privatisé, car les algorithmes d’AA constituen...",French,French
...,...,...,...
8909,inga problem.,Sweedish,French
8440,Bidragsgivare till Wikipedias politikartiklar ...,Sweedish,unknow
6671,"вот попробуйте золотого картофеля, он действит...",Russian,Sweedish
683,"On September 28, 2007, Italian politician Fran...",English,English


# Step - 6 Turning the Model into an Online Application

In [170]:
!pip install -qqq huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [171]:
from huggingface_hub import login
access_token_write = '********'
login(token = access_token_write)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [172]:
trainer.push_to_hub("dancrvlh/language")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/568M [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

'https://huggingface.co/dancrvlh/outputs/tree/main/'

https://huggingface.co/spaces/fastaioncampus/Languages